# 数据处理

In [1]:
import numpy as np
import scipy.io as sio

In [2]:
mat = sio.loadmat('ex8_movies.mat')
mat.keys()
#Y:用户对电影的评分，R：用户是否评分

dict_keys(['__header__', '__version__', '__globals__', 'Y', 'R'])

In [3]:
Y,R=mat['Y'],mat['R']
Y.shape,R.shape

((1682, 943), (1682, 943))

In [4]:
param_mat = sio.loadmat('ex8_movieParams.mat')
param_mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'Theta', 'num_users', 'num_movies', 'num_features'])

In [5]:
X,Theta,nu,nm,nf=param_mat['X'],param_mat['Theta'],param_mat['num_users'],param_mat['num_movies'],param_mat['num_features']

In [6]:
X.shape,Theta.shape,nu,nm,nf

((1682, 10),
 (943, 10),
 array([[943]], dtype=uint16),
 array([[1682]], dtype=uint16),
 array([[10]], dtype=uint8))

In [7]:
#将一维数组转化为数字
nu = int(nu)
nm = int(nm)
nf = int(nf)
nu,nm,nf

(943, 1682, 10)

## 1.序列化参数（由于要用到scipy的优化函数）

In [8]:
def serialize(X,Theta):
    return np.append(X.flatten(),Theta.flatten())

## 2.解序列化参数

In [9]:
def deserialize(params,nm,nu,nf):
    X = params[:nm*nf].reshape(nm,nf)
    Theta = params[nm*nf:].reshape(nu,nf)
    return X,Theta

## 3.代价函数

In [10]:
def costFunction(params,Y,R,nm,nu,nf,lamda):
    X,Theta = deserialize(params,nm,nu,nf)
    error = 0.5 * np.square((X@Theta.T - Y)*R).sum() #*R:点乘R矩阵，也就是说是否有评分，没的话为0，有的话为1
    reg1 = 0.5 * lamda * np.square(X).sum()
    reg2 = 0.5 * lamda * np.square(Theta).sum()
    return error + reg1 + reg2

In [11]:
users = 4
movies = 5
features = 3
#并没有用到全部的数据集，下面取子集
X_sub = X[:movies,:features]
Theta_sub = Theta[:users,:features]
Y_sub = Y[:movies,:users]
R_sub = R[:movies,:users]
cost1 = costFunction(serialize(X_sub,Theta_sub),Y_sub,R_sub,movies,users,features,lamda=0)
cost1

22.224603725685675

In [12]:
cost2 = costFunction(serialize(X_sub,Theta_sub),Y_sub,R_sub,movies,users,features,lamda=1.5)
cost2

31.344056244274217

## 4.梯度 

In [13]:
def costGradient(params,Y,R,nm,nu,nf,lamda):
    X,Theta = deserialize(params,nm,nu,nf)
    X_grad = ((X@Theta.T-Y)*R)@Theta+lamda*X
    Theta_grad = ((X@Theta.T-Y)*R).T@X + lamda*Theta
    return serialize(X_grad,Theta_grad)

## 5.新增一个用户

In [14]:
my_ratings = np.zeros((nm,1))
my_ratings[9] = 5
my_ratings[66] = 5
my_ratings[96] = 5
my_ratings[121] = 4
my_ratings[148] = 4
my_ratings[285] = 3
my_ratings[490] = 4
my_ratings[599] = 4
my_ratings[643] = 4
my_ratings[958] = 5
my_ratings[1117] = 3
Y.shape
my_ratings.shape

(1682, 1)

In [15]:
Y = np.c_[Y,my_ratings]
R = np.c_[R,my_ratings!=0]

In [16]:
Y.shape

(1682, 944)

In [17]:
nm,nu = Y.shape

## 6.均值归一化

In [18]:
def normalizeRatings(Y,R):
    Y_mean = (Y.sum(axis=1)/R.sum(axis=1)).reshape(-1,1)#求均值，然后转换成二维矩阵
    Y_norm = (Y-Y_mean)*R
    return Y_norm,Y_mean

In [19]:
Y_norm,Y_mean= normalizeRatings(Y,R)
Y_norm.shape

(1682, 944)

## 7.参数初始化

In [20]:
X = np.random.random((nm,nf))  #np.random.random()返回的是0-1的浮点数
Theta = np.random.random((nu,nf))
params = serialize(X,Theta)
lamda = 5
R.shape

(1682, 944)

## 8.模型训练

In [28]:
from scipy.optimize import minimize
res = minimize(fun = costFunction,
              x0 = params,
              args = (Y_norm,R,nm,nu,nf,lamda),
              method = 'TNC',
              jac = costGradient,
              options ={'maxiter':100})

In [29]:
params_fit = res.x

In [30]:
fit_X,fit_Theta = deserialize(params_fit,nm,nu,nf)

## 9.预测

In [31]:
Y_pred = fit_X@fit_Theta.T

In [32]:
#排序，找出排名前10的电影
y_pred = Y_pred[:,-1]+Y_mean.flatten()

In [33]:
index = np.argsort(-y_pred) #argsort 是以小到大排序，加了符号就是从大到小排序

In [34]:
index[:10]

array([1535,  813, 1499, 1200, 1598, 1466, 1121, 1188, 1652, 1292],
      dtype=int64)

In [37]:
#处理电影名单，通过空格来进行分割，只要电影名字和电影年份
movies = []
with open('movie_ids.txt','r',encoding='latin 1') as f:
    for line in f:
        tokens = line.strip().split(' ')
        movies.append(''.join(tokens[1:]))

In [38]:
len(movies)

1682

In [39]:
for i in range(10):
    print(index[i],movies[index[i]],y_pred[index[i]])

1535 Aiqingwansui(1994) 5.004183224606328
813 GreatDayinHarlem,A(1994) 5.003885323509188
1499 SantawithMuscles(1996) 5.0037685753971495
1200 MarleneDietrich:ShadowandLight(1996) 5.003485172596662
1598 SomeoneElse'sAmerica(1995) 5.002612734210156
1466 SaintofFortWashington,The(1993) 5.0020862894427065
1121 TheyMadeMeaCriminal(1939) 5.00206809259258
1188 Prefontaine(1997) 5.001536737285158
1652 EntertainingAngels:TheDorothyDayStory(1996) 5.0006562480834855
1292 StarKid(1997) 5.000444724981488
